In [305]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import svm
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.metrics import sensitivity_specificity_support
#from scipy import stats
import math
import statistics as stats

In [306]:
df1 = pd.read_csv("HurstExpCtrl.csv")
row_labels = df1.get('ID').values
df1 = df1.drop(['ID'], axis=1)
column_labels = df1.columns.values

df1 = pd.DataFrame(data=df1.values, index=row_labels, columns=column_labels, dtype=float)

EDSS = df1['EDSS']
data1 = df1.drop(['EDSS'],axis=1)
data1.head()
#print(df.shape)
#df.head()

,MS?,Left-Cerebellum-Cortex,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,Left-VentralDC,...,ctx-rh-rostralanteriorcingulate,ctx-rh-rostralmiddlefrontal,ctx-rh-superiorfrontal,ctx-rh-superiorparietal,ctx-rh-superiortemporal,ctx-rh-supramarginal,ctx-rh-frontalpole,ctx-rh-temporalpole,ctx-rh-transversetemporal,ctx-rh-insula
HC001_20180228.txt,0.0,0.686857,0.674087,0.623960,0.616461,0.594172,0.512860,0.518331,0.670021,0.747563,...,0.623978,0.705369,0.664020,0.814680,0.619224,0.492529,0.929471,0.609840,0.590725,0.565188
HC002_20180312.txt,0.0,0.842905,0.706556,0.637848,0.706745,0.602946,0.613402,0.653011,0.636813,0.664506,...,0.869002,0.607099,0.878091,0.784718,1.035711,0.710424,0.898980,0.811967,0.877609,0.795933
HC003_20180314.txt,0.0,0.746443,0.967989,0.981272,0.746583,1.024804,1.015443,0.978076,0.974523,0.833623,...,1.029552,0.851454,0.787361,0.704076,0.708789,0.899196,0.705697,0.875563,0.505653,0.472877
HC004_20180315.txt,0.0,0.892752,0.752509,0.544667,0.686833,0.747510,0.725371,0.641491,0.470419,0.611579,...,0.928668,0.995723,0.738495,1.086729,0.840650,0.825694,0.836567,0.888991,0.750029,0.803125
HC006_20180426.txt,0.0,0.875850,0.881165,0.736627,0.859676,0.906535,0.601757,0.818481,0.841673,0.780863,...,0.716216,0.786902,0.711456,0.733769,1.051256,0.731897,0.747209,0.533489,0.852048,0.619330


In [307]:
#patient demographic info
df2 = pd.read_csv("HC_Patient_info.csv")
row_labels = df2.get('ID').values
df2 = df2.drop(['ID','Race','Ethnicity'], axis=1)
column_labels = df2.columns.values

df2 = pd.DataFrame(data=df2.values, index=row_labels, columns=column_labels)

#EDSS2 = df2['EDSS']
data2 = df2.drop(['EDSS'],axis=1)


In [308]:
#categorize EDSS <=1.5 is low, >1.5 is high
n,d = data1.shape
#labels = np.where(EDSS<=1.5, 0, 1)
#labels = pd.DataFrame(labels, columns=['EDSS'])

for i in range(n):
    if EDSS[i]<=1.5:
        EDSS[i]=0
    else:
        EDSS[i]=1

for i in range(data2['Gender'].size):
    if data2['Gender'][i] == 'M':
        data2['Gender'][i] = 1
    else:
        data2['Gender'][i] = 0
    #if EDSS2[i]<=1.5:
    #    EDSS2[i]=0;
    #else EDSS2[i]=0
EDSS.head(20)
        

HC001_20180228.txt     0.0
HC002_20180312.txt     0.0
HC003_20180314.txt     0.0
HC004_20180315.txt     0.0
HC006_20180426.txt     0.0
HC007_20180501.txt     0.0
HC008_20180502.txt     0.0
HC009_20180508.txt     0.0
HC010_20180509.txt     0.0
HC011_20180515.txt     0.0
HC012_20180525.txt     0.0
HC013_20180607.txt     0.0
HC014_20180627.txt     0.0
HC015_20180628.txt     0.0
HC016_20180713.txt     0.0
HC017_20180717.txt     0.0
MS0001_20190107.txt    0.0
MS0040_20180727.txt    0.0
MS0046_20170614.txt    0.0
MS0084_20171213.txt    0.0
Name: EDSS, dtype: float64

In [309]:
def best_hp(params, auc):
    
    max_auc = np.amax(auc)
    imax = np.where(auc == max_auc)[0]
    num_max = len(imax)
    
    if num_max%2==0:
        index = np.argmax(auc)
        best_cost = params[index][0]
        best_gamma = params[index][1]
        max_auc = auc[index]
        return best_cost, best_gamma, max_auc

    best_Cs = [0]*num_max
    best_Gs = [0]*num_max
    
    for i in range(num_max):
        index = imax[i]
        best_Cs[i] = params[index][0]
        best_Gs[i] = params[index][1]
    
    try:
        best_cost = stats.mode(best_Cs) #if no unique mode, then smallest of the modes is chosen instead
    except:
        best_cost = stats.median(best_Cs)
        
    try:
        best_gamma = stats.mode(best_Gs) #if no unique mode, then smallest of the modes is chosen instead
    except:
        best_gamma = stats.median(best_Gs)
  
    
    return best_cost, best_gamma, max_auc



In [310]:
def svm_classify(X_train, y_train, X_test, y_test, C, gamma, kernel, cat):
    svm_model = svm.SVC(C=C, kernel=kernel, gamma=gamma, probability=True)
    #svm_bin = svm.SVC(C=C, kernel=kernel, gamma=gamma, probability=False)
    
    if not cat:
        ss=StandardScaler()
        ss.fit(X_train)
        X_train = ss.transform(X_train)
        sm=SMOTE()
        X_train, y_train = sm.fit_resample(X_train, y_train)
    
    #ss.fit(X_test)
        X_test = ss.transform(X_test)
    
    svm_model.fit(X_train, y_train)
    
    pred = svm_model.predict(X_test)
    pred_score = svm_model.predict_proba(X_test)
    
    curve = metrics.roc_curve(y_test, pred_score[:,1])
    #auc = calc_auc(curve)
    
    auc = metrics.roc_auc_score(y_test, pred_score[:,1], average=None)
    bal_acc = metrics.balanced_accuracy_score(y_test, pred)
    sens, spec, _ = sensitivity_specificity_support(y_test, pred, average='binary')
    
    return auc, bal_acc, sens, spec, pred_score[:,1], curve




In [316]:
k_fold = 5
kernel = 'linear'
outer = 5
inner = 2

cost = []
gamma = []
for i in range(-5,5):
    cost.append(10**i)
    #gamma.append(10**i)
    #gamma.append(10**i)
for i in range(-5, 0):
    gamma.append(10**i)
    
#print(gamma)

n,m = len(cost), len(gamma)
params = np.zeros((n*m, 2))

for i in range(n):
    for j in  range(m):
        params[i*m+j][0]=cost[i]
        params[i*m+j][1]=gamma[j]
        
Rkf = model_selection.RepeatedStratifiedKFold(n_splits=k_fold, n_repeats=outer)
kf = model_selection.StratifiedKFold(n_splits=k_fold, shuffle=True)




In [317]:
def run(data1, data2, target, cat):
    results1 = []
    results2 = []
    results3 = []
    #all_results = []
    #auc_curve = []
    
    #split entire data for 5-fold CV
    for train, test in Rkf.split(data1, target):
        X_train1 = data1.iloc[train, :]
        X_test1 = data1.iloc[test, :]
        X_train2 = data2.iloc[train, :]
        X_test2 = data2.iloc[test, :]
        
        y_train = target.iloc[train]
        y_test = target.iloc[test]
        
        inner_par1 = np.zeros((inner,2))
        inner_auc1 = np.zeros(inner)
        inner_par2 = np.zeros((inner,2))
        inner_auc2 = np.zeros(inner)
        
        #inner loop
        for y in range(inner):
            
            count=0
            all_auc1 = np.zeros((n*m, k_fold))
            all_auc2 = np.zeros((n*m, k_fold))
            
            #split for 5-fold CV on inner
            for tr, val in kf.split(X_train1, y_train):
                X_tr1 = X_train1.iloc[tr, :]
                X_val1 = X_train1.iloc[val, :]
                X_tr2 = X_train2.iloc[tr, :]
                X_val2 = X_train2.iloc[val, :]
                
                y_tr = y_train.iloc[tr]
                y_val = y_train.iloc[val]
                #print(y_tr)
                
                for p in range(n*m):
                    C=params[p][0]
                    G=params[p][1]
            
                    auc1, acc1, sen1, _, _, _ = svm_classify(X_tr1, y_tr, X_val1, y_val, C, G, kernel, cat)
                    auc2, acc2, sen2, _, _, _ = svm_classify(X_tr2, y_tr, X_val2, y_val, C, G, kernel, True)
                    all_auc1[p][count] = auc1
                    all_auc2[p][count] = auc2
                count += 1
            
            medians1 = np.median(all_auc1, axis=1)
            medians2 = np.median(all_auc2, axis=1)
            good_C1, good_G1, good_auc1 = best_hp(params, medians1)
            good_C2, good_G2, good_auc2 = best_hp(params, medians2)
            
            inner_par1[y][0] = good_C1
            inner_par1[y][1] = good_G1
            inner_auc1[y] = good_auc1
            inner_par2[y][0] = good_C2
            inner_par2[y][1] = good_G2
            inner_auc2[y] = good_auc2
            #all_results.append([good_C1, good_G1, good_auc1])
            
        best_C1, best_G1, best_auc1 = best_hp(inner_par1, inner_auc1)
        best_C2, best_G2, best_auc2 = best_hp(inner_par2, inner_auc2)
        
        total_auc1, bal_acc1, sens1, spec1, pred1, curve1 = svm_classify(X_train1, y_train, X_test1, y_test, best_C1, best_G1, kernel, cat)
        total_auc2, bal_acc2, sens2, spec2, pred2, curve2 = svm_classify(X_train2, y_train, X_test2, y_test, best_C2, best_G2, kernel, True)
        
        ens_pred_score = [0]*len(pred1)
        ens_preds = [0]*len(pred1)
        
        for i in range(len(pred1)):
            ens_pred_score[i] = (pred1[i] + pred2[i])/2
            if ens_pred_score[i]<.5:
                ens_preds[i] = 0
            else:
                ens_preds[i] = 1
            
        total_auc3 = metrics.roc_auc_score(y_test, ens_pred_score, average=None)
        bal_acc3 = metrics.balanced_accuracy_score(y_test, ens_preds)
        sens3, spec3, _ = sensitivity_specificity_support(y_test, ens_preds, average='binary')
        
        results1.append([best_C1, best_G1, total_auc1, bal_acc1, sens1, spec1])
        results2.append([best_C2, best_G2, total_auc2, bal_acc2, sens2, spec2])
        results3.append([total_auc3, bal_acc3, sens3, spec3])
        
    return results1, results2, results3

In [ ]:
r1, r2, r3 = run(data1, data2, EDSS, False)




In [138]:
med_auc = np.median(r1, axis=0)[2]
med_bacc = np.median(r1, axis=0)[3]
med_sens = np.median(r1, axis=0)[4]
med_spec = np.median(r1, axis=0)[5]
print('Hurst:')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r1), columns=['Cost', 'Gamma', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Hurst:
Median AUC is 0.4431818181818182
Median Balanced Accuracy is 0.5
Median Sensitivity is 0.2
Median Specificity is 0.7


,Cost,Gamma,AUC,Balanced Accuracy,Sensitivity,Specificity
0,10.0,0.00100,0.381818,0.454545,0.00,0.909091
1,10.0,0.00001,0.509091,0.581818,0.80,0.363636
2,10000.0,0.10000,0.400000,0.500000,0.00,1.000000
3,10.0,0.00001,0.450000,0.625000,0.75,0.500000
4,100.0,0.00001,0.375000,0.550000,0.50,0.600000
5,1000.0,0.00010,0.672727,0.563636,0.40,0.727273
6,100.0,0.00010,0.527273,0.409091,0.00,0.818182
7,10.0,0.00100,0.400000,0.518182,0.40,0.636364
8,10.0,0.00001,0.700000,0.450000,0.00,0.900000
9,1000.0,0.00001,0.350000,0.325000,0.25,0.400000


In [139]:
med_auc = np.median(r2, axis=0)[2]
med_bacc = np.median(r2, axis=0)[3]
med_sens = np.median(r2, axis=0)[4]
med_spec = np.median(r2, axis=0)[5]
print('Demographic:')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r2), columns=['Cost', 'Gamma', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Demographic:
Median AUC is 0.5454545454545454
Median Balanced Accuracy is 0.5090909090909091
Median Sensitivity is 0.2
Median Specificity is 0.9090909090909091


,Cost,Gamma,AUC,Balanced Accuracy,Sensitivity,Specificity
0,100.0,0.00010,0.672727,0.563636,0.40,0.727273
1,1.0,0.10000,0.472727,0.500000,0.00,1.000000
2,100.0,0.00100,0.527273,0.509091,0.20,0.818182
3,1.0,0.01000,0.425000,0.575000,0.25,0.900000
4,1.0,0.01000,0.625000,0.650000,0.50,0.800000
5,1.0,0.00001,0.654545,0.500000,0.00,1.000000
6,1.0,0.01000,0.418182,0.500000,0.00,1.000000
7,10000.0,0.00100,0.545455,0.563636,0.40,0.727273
8,1000.0,0.00010,0.525000,0.700000,0.50,0.900000
9,100.0,0.00010,0.775000,0.650000,0.50,0.800000


In [140]:
med_auc = np.median(r3, axis=0)[0]
med_bacc = np.median(r3, axis=0)[1]
med_sens = np.median(r3, axis=0)[2]
med_spec = np.median(r3, axis=0)[3]
print('Ensemble')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r3), columns=['AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Ensemble
Median AUC is 0.5454545454545454
Median Balanced Accuracy is 0.5
Median Sensitivity is 0.0
Median Specificity is 0.9090909090909091


,AUC,Balanced Accuracy,Sensitivity,Specificity
0,0.636364,0.454545,0.00,0.909091
1,0.454545,0.500000,0.00,1.000000
2,0.527273,0.500000,0.00,1.000000
3,0.375000,0.575000,0.25,0.900000
4,0.625000,0.650000,0.50,0.800000
5,0.690909,0.609091,0.40,0.818182
6,0.563636,0.454545,0.00,0.909091
7,0.545455,0.454545,0.00,0.909091
8,0.525000,0.575000,0.25,0.900000
9,0.375000,0.450000,0.00,0.900000


In [303]:
#MS high vs low impairment p-values
#run blocks 2-4 again
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


df1 = df1.sort_values(by='MS?')
df3 = df1.drop(['EDSS', 'MS?'], axis=1)
n,d = df3.shape

split_index = 0
for i in range(n):
    if df1.iat[i, 1]==0:
        split_index +=1

#print(split_index)
low = df3.iloc[:split_index, :]
high = df3.iloc[split_index:, :]
print(p_val)

t_stat, p_val = ttest_ind(high, low, axis=0)
p_val_corrected = multipletests(p_val, method='fdr_bh')[1]
print(p_val_corrected)

[0.83753514 0.53935486 0.73280573 0.12511734 0.8144143  0.96997331
 0.89254107 0.06015219 0.03304356 0.95453255 0.20439074 0.93074109
 0.69694091 0.34056321 0.20204408 0.65061068 0.34464382 0.00772791
 0.82783339 0.32263994 0.55471163 0.86807938 0.7373036  0.19796742
 0.95587136 0.58948491 0.65324274 0.61493565 0.03441481 0.317536
 0.54787462 0.22146713 0.92614283 0.4498124  0.69973218 0.53754554
 0.06936078 0.87494231 0.17759788 0.27705736 0.97622616 0.67789343
 0.5563189  0.22889616 0.23936446 0.48973187 0.96999366 0.79704736
 0.02012448 0.02421243 0.30783405 0.35421736 0.68756647 0.48321028
 0.00986678 0.94613036 0.15736327 0.28485941 0.46319111 0.29444233
 0.53797246 0.08315395 0.29878516 0.42263294 0.09741633 0.87175975
 0.96392102 0.79066118 0.82334048 0.51170022 0.56110395 0.50460073
 0.60537566 0.94074212 0.3173252  0.6160444  0.19313264 0.04003006
 0.00926437 0.6556281  0.53165882 0.48297659 0.0519873  0.32839799
 0.64903543 0.83565742]
[0.50193656 0.90939302 0.73797451 0.9093

In [304]:
print(np.median(p_val))
print(np.median(p_val_corrected))

0.3377595562453498
0.6493226566109289
